# Managing container images

In the previous section of this workshop we've seen how to run container images pulled from different registries but, what if I want to create a container image for my own application? No worries, podman has your back.

Podman can be used for building your own container images and push them to a container registry. In order to do it you need to first build a file called "Containerfile" and then used the "podman build" command to create your image out of the content of the Containerfile.

Lets see an example.

## Creating a new container image

Lets begin by creating a new Container file with the definition of our container image:

In [7]:

!echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY index.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > Containerfile
!cat Containerfile

FROM docker.io/redhat/ubi9
RUN dnf -y update && dnf -y install httpd  && dnf clean all
COPY index.html /var/www/html/index.html
EXPOSE 80
ENTRYPOINT /usr/sbin/httpd -DFOREGROUND


The output of the command is the content of our newly created Containerfile. Let's review it line by line.

First we have specified the "FROM" option, this line is mandatory, you need it to build your container image. With this line we specify the base container image that we will use for our application, in our example we've chosen a redhat/ubi9 image as base for our application. Choosing an image that you can trust is critical to maintain a secure environment, if your base image has vulnerabilities it may affect your application and potentially your whole enviroment. This is why we have chosen a Red Hat Universal Base Image (UBI) image for Red Hat Enterprise Linux 9 (RHEL9).

Red Hat creates UBI images that can be used securely, there are many options for your UBI images, from the standard RHEL 8 or RHEL 9 to a much secure micro image and even images for specific types of applications (like http, nodejs, etc). You can find a detailed list of all the CVE's and security status of Red Hat's UBI images in this website: https://catalog.redhat.com/software/containers/search
Also, if you're a Red Hat customer you will have full support for these images. But, if you're not, do not worry as most of this images are also uploaded to docker.io so you can access them and have enterprise grade security in your containers for free.

With that we now that our application will be running on top of a trusted and secured container image, now we need to add our software layer. To do so we added the second line in which we used the "RUN" option to specify the command that needs to be executed to install the reuqired software in our container image. If you don't have any experience with Fedora or RHEL, we're just using the dnf command to install the httpd package.

Once the httpd package is installed we would like to customized the message that our containerized web server will show to the world. For that we used the "COPY" command. This command will copy a file from our local directory to inside the container that we are building, for our container we will be passing a file index.html from our local directory to the directory /var/www/html/ inside the container. We do this because we now that httpd uses that file to define the content to be posted through our web server.
If we want it to work we will need to create the index.html file so it can the be copied inside the container. Let's do it.

In [8]:
!echo "Hello from HPE Dev and Red Hat teams :)" > index.html
!cat index.html

Hello from HPE Dev and Red Hat teams :)


cat << EOF > Containerfile
FROM docker.io/redhat/ubi9
RUN dnf -y update && dnf -y install httpd  && dnf clean all
COPY index.html /var/www/html/index.html
EXPOSE 80
ENTRYPOINT /usr/sbin/httpd -DFOREGROUND
EOF

cat << EOF > html.index
Hello from HPE Dev and Red Hat teams :)
EOF

podman build --tag rhel9:myhttpd -f ./Containerfile


podman run -d -p 8080:80 --name myhttpd --rm rhel9:myhttpd

curl localhost:8080